In [334]:
import pyreadr

train = pyreadr.read_r('data_train.rds') # also works for RData
test = pyreadr.read_r('data_test.rds') # also works for RData



In [335]:
import pandas as pd
import gc
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import time
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, accuracy_score, plot_roc_curve
from sklearn.preprocessing import MaxAbsScaler
import matplotlib.pyplot as plt

In [336]:
train_df = train[None]
test_df = test[None]

In [337]:
mapping = {'M': 1, 'F': 0}
train_df.replace({'sex': mapping},inplace=True)
test_df.replace({'sex': mapping},inplace=True)

mapping2 = {'AD': 0, 'SC': 1}
train_df.replace({'histology': mapping2},inplace=True)

In [338]:
mapping3 = {False: 0, True: 1}
train_df.replace({'dead': mapping3},inplace=True)

In [339]:
mapping4 = {'FALSE': 0, 'TRUE': 1, np.nan :0}
train_df.replace({'dead_at_censor_months': mapping4},inplace=True)

In [340]:
train_df['dead_at_censor_months'].value_counts()

0    364
1     44
Name: dead_at_censor_months, dtype: int64

In [341]:
labels = train_df['dead_at_censor_months']

In [342]:
train_cont = train_df.iloc[:,6:].copy()
test_cont = test_df.iloc[:,6:].copy()

In [343]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit on training set only.
scaler.fit(train_cont)
# Apply transform to both the training set and the test set.
train_cont = scaler.transform(train_cont)
test_cont = scaler.transform(test_cont)

In [344]:
from sklearn.decomposition import PCA
# Make an instance of the Model

pca = PCA(.97)
pca.fit(train_cont)
train_cont = pca.transform(train_cont)
test_cont = pca.transform(test_cont)

In [345]:
train_df['age2'] = train_df['age']
test_df['age2'] = test_df['age']
scaler.fit(train_df[['age','age2']])
# Apply transform to both the training set and the test set.
train_oth = scaler.transform(train_df[['age','age2']])
test_oth = scaler.transform(test_df[['age','age2']])

In [346]:
train_oth[:,1]=train_df['sex'].values
test_oth[:,1]=test_df['sex'].values

In [347]:
from scipy import stats
def T_Test(features,labels):
    t_test = [] #prealocate
    for y in range(0,np.shape(features)[1]):
        t_test.append(stats.ttest_ind(features[np.where(labels==1),y][0],features[np.where(labels==0),y][0])[0]) # perform t-test for each feature and record result
    t_test = np.absolute(np.array(t_test)) #take absoolute values of t-test statistic
    feature_set = t_test.argsort()[::-1][:60] # find rank idexes from lowest to highest, reverse an array and take 10 first elements - indecies of 10 highest t statistics
    return feature_set

In [348]:
test_df.isna().sum().sum()

300

In [349]:
feature_set = T_Test(features=train_cont,labels=labels)

In [350]:
train_cont = train_cont[:,feature_set]
test_cont = test_cont[:,feature_set]

In [351]:
X_train = np.hstack((train_oth,train_cont))
X_test = np.hstack((test_oth,test_cont))

In [353]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# create model
model = SVC(kernel="rbf",class_weight='balanced')
#model = RandomForestClassifier(n_estimators=100,
#                               class_weight = {0:0.89215, 1: 0.10784})
#                               class_weight='balanced')
                               #max_features = 'sqrt')
# evaluate model
scores = cross_val_score(model, X_train, labels, scoring='f1', cv=cv, n_jobs=-1) #'roc_auc'
# report performance
print('f1: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))
model.fit(X_train, labels)

# Get predictions on the test set
y_pred = model.predict(X_test)

f1: 0.504 (0.250)


In [354]:
df = pd.DataFrame(y_pred, 
             columns=['predictions'])

In [355]:
mapping5 = {0:'FALSE', 1:'TRUE'}
df.replace({'predictions': mapping5},inplace=True)

In [356]:
df.value_counts()

predictions
FALSE          73
TRUE           27
dtype: int64

In [357]:
df.to_csv('results.txt', index=False,header=False)